In [1]:
import sys
!{sys.executable} -m pip install scikit-learn==0.22.2.post1
import warnings 
warnings.filterwarnings('ignore')

  Using cached scikit-learn-0.22.2.post1.tar.gz (6.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for scikit-learn
Failed to build scikit-learn
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.1
    Uninstalling scikit-learn-1.1.1:
      Successfully uninstalled scikit-learn-1.1.1
  Running setup.py install for scikit-learn: started
  Running setup.py install for scikit-learn: finished with status 'error'
  Rolling back uninstall of scikit-learn
  Moving to c:\users\valen\appdata\local\programs\python\python310\lib\site-packages\scikit_learn-1.1.1.dist-info\
   from C:\Users\valen\AppData\Local\Programs\Python\Python310\Lib\site-packages\~cikit_learn-1.1.1.dist-info
  Moving to c:\users\valen\appdata\local\programs\python\python310\lib\site-packages\sklearn\
   from C:\Users\valen\AppData\Local\Programs\Python\Python310\Lib\site-packages\~klearn


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [33 lines of output]
  Partial import of sklearn during the build process.
  C:\Users\valen\AppData\Local\Temp\pip-install-3sboxpr4\scikit-learn_73d0d3d6b015401a87b67f8cbd88afbc\setup.py:123: DeprecationWarning:
  
    `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
    of the deprecation of `distutils` itself. It will be removed for
    Python >= 3.12. For older Python versions it will remain present.
    It is recommended to use `setuptools < 60.0` for those Python versions.
    For more details, see:
      https://numpy.org/devdocs/reference/distutils_status_migration.html
  
  
    from numpy.distutils.command.build_ext import build_ext  # noqa
  Traceback (most recent call last):
    File "C:\Users\valen\AppData\Local\Temp\pip-install-3sboxpr4\scikit-learn_73d0d3d6b015401a87b67f8cbd88afbc\sklearn\_build_utils\__init__.py", line 32, in _check_cython_v

In [2]:
!{sys.executable} -m pip install sklearn-genetic

In [3]:
!{sys.executable} -m pip install mlxtend

In [4]:
import pandas as pd
import numpy as np
import joblib
import sklearn

sklearn.externals.joblib = joblib

from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, train_test_split
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import classification_report


credit = pd.read_csv('creditcard.csv')
X = credit.loc[:, credit.columns != 'Class']
y = credit['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [5]:
credit.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## 0. Baseline 

In [6]:
# Build full model with selected features on the train dataset
clf = LogisticRegression(C=10, max_iter=10000)
clf.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=10000)

In [7]:
# Test the build model on the testing dataset
y_test_pred = clf.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.68      0.75        98

    accuracy                           1.00     56962
   macro avg       0.91      0.84      0.87     56962
weighted avg       1.00      1.00      1.00     56962



## 1. Forward Selection

In [8]:
scv = StratifiedKFold(n_splits=5)

model = LogisticRegression(C=10, max_iter=10000)

# create the SequentialFeatureSelector object
sfs = SequentialFeatureSelector(model, 
                 k_features=8, 
                 forward=True, 
                 floating=False,
                 verbose=2,               
                 scoring='f1_macro',
                 cv=scv,
                 n_jobs=-1)

# fit the object to the training data
sfs = sfs.fit(X, y)
selected_features = credit.columns[list(sfs.k_feature_idx_)]
print(selected_features)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    6.9s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.3s finished

[2023-01-09 20:55:00] Features: 1/8 -- score: 0.7865019286689792[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  29 | elapsed:    5.0s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done  29 out of  29 | elapsed:    7.7s finished

[2023-01-09 20:55:08] Features: 2/8 -- score: 0.8297761567778965[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  28 | elapsed:    5.3s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  28 out of  28 | elapsed:    8.3s finished

[2023-01-09 20:55:17] Features: 3/8 -- score: 0.8428746273691121[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  2

Index(['V5', 'V8', 'V10', 'V12', 'V14', 'V16', 'V21', 'V23'], dtype='object')



[2023-01-09 20:56:32] Features: 8/8 -- score: 0.8560062048830573

In [9]:
X_train[selected_features]

,V5,V8,V10,V12,V14,V16,V21,V23
265518,1.502822,1.139880,-0.157403,0.510277,-0.066555,0.352514,0.076197,0.307915
180305,2.943487,0.674782,0.284864,0.325560,0.721068,-0.754029,0.038628,0.035542
42664,-0.825838,-5.037523,-2.009561,1.820161,0.122746,1.123344,-2.798352,-0.436530
198723,-1.394143,0.010010,1.756121,-0.722450,-0.195288,-0.065888,-0.139670,0.208310
82325,0.715771,-0.113817,-0.558955,-0.054708,0.134659,-2.096099,-0.243245,-0.006692
...,...,...,...,...,...,...,...,...
233802,-0.285128,0.231591,0.212619,0.584204,0.062302,0.684356,0.262526,0.099141
85418,0.584698,0.135561,0.108640,0.373473,-0.401809,-1.879937,-0.072452,0.110048
29062,-0.380277,-0.026687,-0.131553,0.445453,0.166727,-0.796913,-0.061991,-0.050485
13766,0.065052,-0.091833,-0.268878,-1.666023,1.799882,0.705997,-0.409934,0.033153


In [10]:
# Build full model with selected features on the train dataset
clf = LogisticRegression(C=10, max_iter=10000)
clf.fit(X_train[selected_features], y_train)

LogisticRegression(C=10, max_iter=10000)

In [11]:
# Test the build model on the testing dataset
y_test_pred = clf.predict(X_test[selected_features])
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.61      0.71        98

    accuracy                           1.00     56962
   macro avg       0.92      0.81      0.85     56962
weighted avg       1.00      1.00      1.00     56962



## 2. Genetic Algorithm

A Genetic Algorithm (GA) is a metaheuristic algorithm  inspired by the process of natural selection that belongs to the larger class of evolutionary algorithms (EA). Genetic algorithms are commonly used to generate high-quality solutions to optimization and search problems by relying on biologically inspired operators such as mutation, crossover and selection.

Short story: An algortim inspited from genetics and natural selection which determines the fittest of individuals from a population.

**Steps of the genetic algortihm:**

*1. Initialization* - 
Start with an arbitrary number of different subsets of features from a total number pf N features. The representation: a binary vector of dim = number of features. 1 indicates presence of feature, while 0 absence. For example, the vector [1,0,0,1] indicates that we have in total 4 features and this member of the population is a subset of feature 0 and 3.


---



*2. Fitness Assignment* - 
In order to evaluate each member of our population, we train N models, each with a member from the population and calculate the error. If the error is large then the fitness is low. All errors of the individual N models are sorted but the fitness value depends squarely on the position of the individual in the rank and not at the error.


---



*3. Selection* - 
We use a method that places all members of the population on a roulette, with areas proportional to their fitness. Then we select half of the population at random by “turning the roulette”.


---



*4. Crossover Operation* - 
We choose 2 members randomly and combine their features to get 4 offsprings for the new population. Then we repeat with other 2 members and so on. When do we stop? When we generate the same size of population as in the initial step.


---

*5. Mutation Operation* - 
This step exists because of the latter. During Crossover there is a high chance of generating offsprings that are very similar to their ancestors, which leads to low diversity. Thus, we mutate the offsprings by changing the value of some features of the offsprings at random.You repeat till the next generation does not significantly differ from the previous one. Then you select the member that produced the lowest error.

In [12]:
scv = StratifiedKFold(n_splits=5)
selector = GeneticSelectionCV(LogisticRegression(C=10, max_iter=10000, random_state=42),
                              cv=scv,
                              verbose=2,
                              scoring="f1_macro",
                              max_features=5,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=4,  # recomended 40 or above
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.697384  3.02      0.019725]	[ 0.107111  1.348925  0.01844 ]	[ 0.499567  1.        0.000002]	[ 0.83758   5.        0.117245]
1  	25    	[-1399.336855     3.96      1400.015609]	[ 3470.137876     1.4827    3469.864017]	[-10000.            1.            0.009923]	[     0.849678      8.        10000.      ]
2  	31    	[-1399.314287     4.14      1400.014506]	[ 3470.146982     1.371277  3469.864462]	[-10000.      1.      0.]                  	[     0.854643      8.        10000.      ]
3  	30    	[-799.234398    4.28      800.014872]   	[ 2713.157757     1.020588  2712.927608]	[-10000.            2.            0.011038]	[     0.854643      7.        10000.      ]
4  	30    	[-799.221196    4.56      800.013918]   	[ 2713.16165      0.962497  2712.927889]	[-10000.            2.            0.011511]

In [13]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]

# train and test
clf = LogisticRegression(C=10, max_iter=10000, random_state=42)
clf.fit(X_train[selected_feats], y_train)
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.82      0.57      0.67        98

    accuracy                           1.00     56962
   macro avg       0.91      0.79      0.84     56962
weighted avg       1.00      1.00      1.00     56962

